In [1]:
from pathlib import Path
import json
import inspect
from pprint import pprint

from nwb_conversion_tools import spec

# Specify a field in some external file.

These classes inherit from `spec.external_file.BaseExternalFileSpec`, and allow you to specify some metadata `key` with a value pulled from some potentially nested `field` in a `path` relative to the `base_path` passed during `parse`

start by making some fake metadata and saving it as, say, .json

In [2]:
metadata = {
    'subject': {
        'id': 'jonny',
        'dob': 'not tellin u nosy',
        
    },
    'timestamp': '2021-03-idk-today',
    'stimuli': {
        'left_tone':{
            'hz': 10000,
            'amplitudes': [1,2,4,5]
        },
        'right_tone':{
            'hz': 5000,
            'amplitudes': [6,7,8,9]
        }
    }
}

# save to 'subdirectory_1/subdirectory_2/fake_metadata.json'
temp_json_file = Path() / 'subdirectory_1' / 'subdirectory_2'
temp_json_file.mkdir(parents=True, exist_ok=True)
temp_json_file = temp_json_file / 'fake_metadata.json'
with open(temp_json_file, 'w') as jfile:
    json.dump(metadata, jfile)

Now we specify that, relative to some directory (we want to deploy the same pattern over a number of directories),
how to find the file and index it

In [3]:
json_spec = spec.JSON(
    path=Path('subdirectory_2/fake_metadata.json'), 
    key="Subject ID", 
    field = ('subject', 'id')
)

The NWBConverter class will then, when run_conversion is called, parse the spec with the current directory as the argument

In [4]:
json_spec.parse(Path('subdirectory_1'))

{'Subject ID': 'jonny'}

It'll slice however you want (including i guess if you want to pass `slice` objects)

In [5]:
json_spec_2 = spec.JSON(
    path=Path('subdirectory_2/fake_metadata.json'), 
    key="Stimulus Amplitude", 
    field = ('stimuli', 'left_tone', 'amplitudes', 2)
)

In [6]:
json_spec_2.parse(Path('subdirectory_1'))

{'Stimulus Amplitude': 4}

# Extending

You should only need to override the `_load_data` method with one that takes a path and returns a potentially nested dictionary.

See, for example, the `JSON` class's docstring is about as long as all the code include boilerplate:

In [8]:
print(inspect.getsource(spec.JSON))

class JSON(BaseExternalFileSpec):

    def __init__(self, hook:typing.Optional[typing.Callable]=None,
                 *args, **kwargs):
        """
        Load a field from a .json file. see base class for docs

        Parameters
        ----------
        hook : Optionally, include some callable function to use as the fallback
            object loader hook (see ``object_hook`` argument in ``json.load`` for more information)
        args : passed to :class:`.BaseExternalFileSpec`
        kwargs :
        """
        self.hook = hook
        super(JSON, self).__init__(*args, **kwargs)

    def _load_file(self, path:Path) -> dict:
        with open(path, 'r') as p:
            loaded = json.load(p, object_hook=self.hook)
        return loaded



(clean up after ourselves)

In [11]:
temp_json_file.parents[1].rmdir()

OSError: [Errno 66] Directory not empty: 'subdirectory_1'